<a href="https://colab.research.google.com/github/JohnTaylor2000/Machine-learning/blob/master/ConvLSTM2d_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Title**: Next-frame prediction with Conv-LSTM
**Author**: [jeammimi](https://github.com/jeammimi)
**Date created**: 2016/11/02
**Last modified:** 2020/06/01

**Description:** Predict the next frame in a sequence using a Conv-LSTM model.

**Adapted:** by John Taylor 2020/07 to run in google colab and have added back plotting

## Introduction

This script demonstrates the use of a convolutional LSTM model.
The model is used to predict the next frame of an artificially
generated movie which contains moving squares.


Start by loading **Nvidia GPU environment** variables that suppress warning messages. Only needed when running on nvidia GPUs.

In [ ]:
import os
os.environ["TF_DISABLE_NVTX_RANGES"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["NCCL_DEBUG"] = "WARN"



Add **Matplotlib** plotting library and **Numpy** maths library

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

Add the **time** library so we can time code

In [ ]:
import time

Add **TensorFlow** - we will use Keras to create the model.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, Conv3D

**Test for the presence of a GPU device**

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

**Build the model**

We create a model which take as input movies of shape
`(n_frames, width, height, channels)` and returns a movie
of identical shape.
We set Variable-length sequence of 40x40x1 frames

Select the **frame size, number of filters and layers** in the model, then define and compile the model.

In [ ]:
frame_size = 40     # number of pixels in x and y dimensions
num_filters = 40    # base number of hidden units
num_layers = 5      # in addition to input and output layer
scale_fac = 1       # increase the number of hidden units per layer

model = keras.Sequential()

model.add(Input(shape=(None, frame_size, frame_size, 1)))

scale = scale_fac
if num_layers > 0:
  scale = scale_fac
  for i in range (num_layers):
    model.add(ConvLSTM2D(filters=num_filters*scale, kernel_size=(3, 3), padding="same", return_sequences=True))
    model.add(BatchNormalization())
    scale = scale * scale_fac

# must set first parameter in kernel size = 1 to track movement

model.add(Conv3D(filters=1, kernel_size=(1, 3, 3), activation="sigmoid", padding="same"))
  
model.compile(loss="binary_crossentropy", optimizer="adam")
#model.compile(loss="mse", optimizer="adam")



**Print a summary of the model**

In [ ]:
model.summary()

**Generate artificial data**

Generate movies with 3 to 7 moving squares inside.
The squares are of shape 1x1 or 2x2 pixels,
and move linearly over time.

For convenience, we first create movies with bigger width and height (80x80)
and at the end we select a 40x40 window.


In [ ]:
def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)

    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(5, 10)

        for j in range(n):
            # Initial position
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # Direction of motion
           # directionx = np.random.randint(0, 3) - 1
            directionx = np.random.randint(0, 5) - 1
            directiony = np.random.randint(0, 5) - 1

            # Size of the square
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t
                noisy_movies[
                    i, t, x_shift - w : x_shift + w, y_shift - w : y_shift + w, 0
                ] += 1

                # Make it more robust by adding noise.
                # The idea is that if during inference,
                # the value of the pixel is not exactly one,
                # we need to train the model to be robust and still
                # consider it as a pixel belonging to a square.
                noise = False
                if np.random.randint(0, 2) and noise:
                    noise_f = (-1) ** np.random.randint(0, 2)
                    noisy_movies[
                        i,
                        t,
                        x_shift - w - 1 : x_shift + w + 1,
                        y_shift - w - 1 : y_shift + w + 1,
                        0,
                    ] += (noise_f * 0.1)

                # Shift the ground truth by 1
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[
                    i, t, x_shift - w : x_shift + w, y_shift - w : y_shift + w, 0
                ] += 1

    # Cut to a 40x40 window
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    
    return noisy_movies, shifted_movies

#noisy_movies, shifted_movies = generate_movies(n_samples)
#for i in range(40):
#  print (noisy_movies[1,1,i,:40,0])


**Train the model**

In [ ]:
num_epochs     = 10  # In practice, you would need hundreds of epochs.
n_samples  = 1200 
batch_size = 10


t0 = time.time()

noisy_movies, shifted_movies = generate_movies(n_samples)

history = model.fit(
    noisy_movies[:1000],
    shifted_movies[:1000],
    batch_size=batch_size,
    epochs=num_epochs,
    verbose=1,
    validation_split=0.1,
)
elapsed_time = time.time() - t0
print (' Model Train Elapsed Time (sec) = ', elapsed_time)
print (' Completed ', num_epochs, ' Epochs.', ' Elapsed Time (sec) = ', elapsed_time)


**Plot the train and test model history**

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

**Test the model on one movie**

Feed it with the first 7 positions and then
predict the new positions.

In [ ]:
movie_index = 1004
test_movie = noisy_movies[movie_index]

# Start from first 7 frames
track = test_movie[:7, ::, ::, ::]

# Predict 16 frames
for j in range(16):
    print(j)
    new_pos = model.predict(track[np.newaxis, ::, ::, ::, ::])
    #print('new_pos = ', new_pos.shape)
    new = new_pos[::, -1, ::, ::, ::]
    track = np.concatenate((track, new), axis=0)
    #print('track = ', track.shape)

print(track.shape)

track2 = noisy_movies[movie_index][::, ::, ::, ::]
for i in range(15):
    fig = plt.figure(figsize=(20, 10))

    ax = fig.add_subplot(121)

    if i >= 7:
        ax.text(1, 3, 'Predictions !', fontsize=20, color='w')
    else:
        ax.text(1, 3, 'Initial trajectory', fontsize=20, color='w')

    toplot = track[i, ::, ::, 0]

    plt.imshow(toplot, cmap="cool")
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', color='w', fontsize=20)

    toplot = track2[i, ::, ::, 0]
    if i >= 2:
        toplot = shifted_movies[movie_index][i - 1, ::, ::, 0]
    plt.imshow(toplot)
    # plt.savefig('%i_animate.png' % (i + 1))
